# Master TIDE - Conférences Python 2020

Francis Wolinski

&copy; 2020 Yotta Conseil

# 9. Transformation des données

## 9.1 Index et colonnes hiérarchiques
La librairie **pandas** permet de gérer des index et des colonnes hiérarchiques. Dans ce cas, les clefs d'accès sont des tuples (ou bien la clef seule pour le premier niveau).

### Index hiérarchique ou multi-index
Le `DataFrame` des communes comporte implicitement 3 niveaux d'index : `Région`, `Département` et `Code INSEE` (ou le nom de la commune).

In [ ]:
# import des modules usuels
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# commande magique pour l'affichage des graphiques
%matplotlib inline

# options d'affichage
pd.set_option("display.max_rows", 16)
plt.style.use('seaborn-darkgrid')

In [ ]:
# chargement des données
geo = pd.read_csv("correspondance-code-insee-code-postal.csv",
                   sep=';',
                   usecols=range(11),
                  index_col="Code INSEE")
geo= geo.sort_index()
geo.head()

In [ ]:
geo["Latitude"] = geo["geo_point_2d"].apply(lambda x: float(x.split(', ')[0]))
geo["Longitude"] = geo["geo_point_2d"].apply(lambda x: float(x.split(', ')[1]))
geo["Superficie"] /= 100.0
geo["Densité"] = 1000 * geo["Population"] / geo["Superficie"]

# conversion de la colonne Statut en catégorie
# selon version de pandas...

statuts = ["Commune simple", "Chef-lieu canton", "Sous-préfecture",
            "Préfecture", "Préfecture de région", "Capitale d'état"]

from pandas.api.types import CategoricalDtype
cat_statut = CategoricalDtype(categories=statuts, ordered=True)
geo["Statut"] = geo["Statut"].astype(cat_statut)

In [ ]:
# index de geo
geo.index

On annule l'index existant avec la fonction **reset_index()** :

In [ ]:
# annulation de l'index existant
geo1 = geo.reset_index()
geo1.head()

In [ ]:
# type de geo1.index
geo1.index

In [ ]:
# on applique un index triple : Région, Département, Code INSEE
geo1 = geo1.set_index(["Région", "Département", "Code INSEE"])
geo1

In [ ]:
# l'index est devenu un MultiIndex
type(geo1.index)

In [ ]:
# geo1.index
geo1.index

Accès direct au premier niveau :

In [ ]:
# premier niveau
geo1.loc[("ILE-DE-FRANCE",)]  # ou bien communes1.loc["ILE-DE-FRANCE"]

Accès au deuxième niveau :

In [ ]:
# deuxième niveau
geo1.loc[("ILE-DE-FRANCE", "HAUTS-DE-SEINE")]

Accès au troisième niveau :

In [ ]:
# troisième niveau
geo1.loc[("ILE-DE-FRANCE", "HAUTS-DE-SEINE", "92002")]

Tri du multi-index pour éviter les problèmes de performance.

In [ ]:
# tri du multi-index
geo1 = geo1.sort_index()
geo1.loc[("ILE-DE-FRANCE", "HAUTS-DE-SEINE")]

### Colonnes hiérarchiques

In [ ]:
import os

if os.path.exists('thd.dat'):
    thd = pd.read_pickle('thd.dat')
else:
    thd = pd.read_excel("FranceTHD_Open_Data_Observatoire_Juin2015.xlsx",
                        sheet_name="Communes",
                        header=1,
                        index_col="Code INSEE",
                        names=["Département", "Code INSEE", "Commune",
                                    "1 Mbit", "3 Mbit", "8 Mbit", "30 Mbit", "100 Mbit",
                                    "DSL 1 Mbit", "DSL 3 Mbit", "DSL 8 Mbit", "DSL 30 Mbit", "DSL 100 Mbit",
                                    "Câble 1 Mbit", "Câble 3 Mbit", "Câble 8 Mbit", "Câble 30 Mbit", "Câble 100 Mbit",
                                    "Fibre 1 Mbit", "Fibre 3 Mbit", "Fibre 8 Mbit", "Fibre 30 Mbit", "Fibre 100 Mbit"])
    thd.to_pickle('thd.dat')

thd.head()

In [ ]:
# on restreint les données au haut débit seul
thd2 = thd.loc[:, "1 Mbit":]
thd2.head()

On constate qu'il y a implicitement 2 niveaux de colonnes : `Techno` et `Débit` (ce qu'il y avait dans le fichier Excel initial).

In [ ]:
# on crée des colonnes hiérarchiques que l'on nomme également
thd2.columns = [["THD", "THD", "THD", "THD", "THD",
                     "DSL", "DSL", "DSL", "DSL", "DSL",
                     "Câble", "Câble", "Câble", "Câble", "Câble",
                     "Fibre", "Fibre", "Fibre", "Fibre", "Fibre"],
                   ["1 Mbit", "3 Mbit", "8 Mbit", "30 Mbit", "100 Mbit",
                   "1 Mbit", "3 Mbit", "8 Mbit", "30 Mbit", "100 Mbit",
                   "1 Mbit", "3 Mbit", "8 Mbit", "30 Mbit", "100 Mbit",
                   "1 Mbit", "3 Mbit", "8 Mbit", "30 Mbit", "100 Mbit"]]
thd2.columns.names = ["Techno", "Débit"]  # on nomme les 2 index"
thd2.head()

On retrouve la structure du fichier Excel initial avec des cellules fusionnées.

In [ ]:
# accès à une valeur
thd2.loc["01002", ("DSL", "3 Mbit")]

In [ ]:
# accès à une portion de ligne
thd2.loc["01002", ("DSL")]  # ou thd2.loc["01001", "DSL"]

**Remarque** : La liste des colonnes hiérarchiques comporte des répétitions. Il aurait été possible d'utiliser les opérateurs `+` et `*` qui agissent sur les listes :

In [ ]:
[["THD"] * 5 + ["DSL"] * 5 + ["Câble"] * 5 + ["Fibre"] * 5,
 ["1 Mbit", "3 Mbit", "8 Mbit", "30 Mbit", "100 Mbit"] * 4]

## 9.2 Permutation de niveaux hiérarchiques

Il est possible d'échanger des niveaux hiérarchiques des index ou des colonnes avec la méthode `swaplevel()` :

In [ ]:
# head
thd2.head()

In [ ]:
# swaplevel
thd2.swaplevel("Débit", "Techno", axis=1).head()

Les niveaux sont bien inversés mais les débits identiques sont éclatés. On verra ci-après comment les rassembler.

Les méthodes `stack()` et `unstack()` permettent d'empiler les colonnes en index et de dépiler les index en colonnes.

In [ ]:
# head
thd2.head()

La méthode **stack()** prend le niveau de colonne le plus bas et l'envoie au niveau d'index le plus bas.

In [ ]:
# on déplace le niveau de colonne "Débit" vers les index
thd2.stack()

On peut répéter l'opération jusqu'à épuisement des niveaux de colonnes. A la fin on obtient un objet `Series` avec un index hiérarchique.

In [ ]:
# on déplace les niveau de colonne "Débit" et "Techno" vers les index
thd2.stack().stack()

In [ ]:
# on peut ensuite faire un reset_index qui ramène tous les index en colonnes
thd2.stack().stack().reset_index()

Inversement, la méthode **unstack()** prend le niveau d'index le plus bas et l'envoie au niveau de colonne le plus bas.

In [ ]:
# unstack
thd2.stack().unstack()

Il est possible de préciser le niveau que l'on souhaite empiler ou dépiler : soit en indiquant le nom de l'index ou de la colonne soit en précisant son rang.

<div class="alert alert-success">
<b>Exercice 2</b>
<ul>
    <li>Trouvez un moyen pour obtenir en colonne les débits rassemblés puis les technos.</li>
</ul>

## 9.3 Mise à plat des données

La fonction `melt()` permet de passer d'un format large (nombreuses colonnes, par ex. tableau de bord pour les humains) à un format long (nombreuses lignes, par ex. table d'une base de données), en précisant :
- le `DataFrame`
- les colonnes d'index, qui restent inchangées
- les colonnes de valeurs, dont les noms deviendront les valeurs d'une colonne nommée `variable` et les valeurs deviendront les valeurs d'une colonne nommée  `value`.

In [ ]:
# suppresion de l'index
thd = thd.reset_index()
thd.head()

In [ ]:
# utilisation de la fonction melt()
var = pd.melt(thd,
         id_vars=['Code INSEE', 'Département', 'Commune'],
         value_vars=["1 Mbit", "3 Mbit", "8 Mbit", "30 Mbit", "100 Mbit",
                                "DSL 1 Mbit", "DSL 3 Mbit", "DSL 8 Mbit", "DSL 30 Mbit", "DSL 100 Mbit",
                                "Câble 1 Mbit", "Câble 3 Mbit", "Câble 8 Mbit", "Câble 30 Mbit", "Câble 100 Mbit",
                                "Fibre 1 Mbit", "Fibre 3 Mbit", "Fibre 8 Mbit", "Fibre 30 Mbit", "Fibre 100 Mbit"])
var

In [ ]:
var.sort_values(['Code INSEE', 'variable']).head(16)

On vérifie la taille du nouveau `DataFrame`.

In [ ]:
# DataFrame initial
thd.shape

In [ ]:
# DataFrame final : nombre de communes x nombre de débits (20)
thd.shape[0] * 20

In [ ]:
# accès à une ligne particulière
var.loc[(var['Commune'] == 'Toulouse') & (var['variable'] == '100 Mbit')]

## 9.4 Fusion d'objets de type `DataFrame`

Dans cette section nous allons fusionner le `DataFrame` des communes avec celui des données sur le haut débit. En utilisant l'index "`Code INSEE`" comme pivot.

La fonction `merge()` permet de fusionner 2 objets de type `DataFrame` selon une colonne de chaque `DataFrame`.

Il existe une option "`how`" pour spécifier comment s'effectue la jointure (terme utilisé dans les base de données) :
- left (par défaut) : utilise la colonne du `DataFrame` de gauche
- right : utilise l'index du `DataFrame` de droite
- outer : forme l'union des index
- inner : utilise l'intersection des index

Il existe aussi les options `suffixes` pour suffixer les colonnes qui seraient dupliquées entre les 2 `DataFrames`, par défaut : `_x` et `_y`.

In [ ]:
# geo head
geo = geo.reset_index()
geo.head()

In [ ]:
# thd head
thd.head()

In [ ]:
alldata = pd.merge(geo, thd, on='Code INSEE', how='left')
alldata

In [ ]:
alldata[['Commune_x', 'Commune_y']]

In [ ]:
# exemple avec suffixes
alldata = pd.merge(geo, thd, on='Code INSEE', how='left', suffixes=('', '_'))
alldata.columns

On constate que *pandas* a suffixé les 2 colonnes "Département" et "Commune" du *DataFrame* de gauche.

Il existe plusieurs autres fonctions de fusion de *DataFrame*.

- méthode `join()` : jointure sur index ou colonne (équivalent à la fonction `merge()`)
- fonction `concat()` : concaténation de `DataFrames` selon un axe
- méthode `append()` : ajout d'un objet `Series` ou `DataFrame` (équivalent à la fonction `concat()`)

Voir la documentation : http://pandas.pydata.org/pandas-docs/stable/merging.html

<div class="alert alert-success">
<b>Exercice 3</b>
<ul>
    <li>Construire une table pivot avec les données allant du 1 Mbit au 100 Mbit ventilées par statut des communes.</li>
</ul>